In [1]:
import torch
import os
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
class MelodyLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, device):
        super(MelodyLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.embedding = torch.nn.Embedding(num_embeddings=)
        
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                                  num_layers=num_layers, batch_first=True)
        
        self.fc_1 = torch.nn.Linear(hidden_size, 256)
        self.fc = torch.nn.Linear(256, output_size)
        
        self.sigmoid = torch.nn.Sigmoid()
        self.threshold = torch.nn.Threshold()
        self.device = device
        
    def forward(self, x):
        x.to(self.device)
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, requires_grad=True).to(self.device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, requires_grad=True).to(self.device)
        
        # output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn, _ = self.lstm(x)
        
        #hn = hn.view(-1, self.hidden_size)
        hn = hn[:, -1, :]
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        
        return out

## Test Data

In [3]:
ub = 84
lb = 24

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [5]:
class ChordMelodyDataset(torch.utils.data.Dataset):
    def __init__(self, size, X_path, y_path, window_size=10):
        self.size = size
        print('Loading features...')
        self.X = pd.read_csv(X_path, nrows=size)
        print('Loading labels...')
        self.y = pd.read_csv(y_path, nrows=size)
        self.window_size = window_size
        
    def __len__(self):
        return len(self.X.index) - self.window_size
    
    def __getitem__(self, idx):
        x_data = torch.tensor(self.X.iloc[idx : idx + self.window_size].to_numpy(dtype=float))
        y_data = torch.tensor(self.y.iloc[idx : idx + self.window_size].to_numpy(dtype=float))
        # x_data = torch.reshape(x_data, (x_data.shape[0], 1, x_data.shape[1]))
        return x_data.float(), y_data.float()
        

In [24]:
dataset = ChordMelodyDataset(300000, '../data/X.csv', '../data/y.csv')

Loading features...
Loading labels...


In [25]:
dataset[0][0].shape

torch.Size([10, 111])

In [26]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=64)

In [27]:
for X, y in data_loader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([64, 10, 111])
torch.Size([64, 10, 60])


## Training

In [28]:
num_epochs = 100
learning_rate = 1e-6

input_size = 111 #number of features
hidden_size = 512 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

output_size = 60 #number of output classes 

In [29]:
mlstm = MelodyLSTM(input_size, hidden_size, output_size, num_layers, device)
mlstm = mlstm.to(device)

In [30]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(mlstm.parameters(), lr=learning_rate) 

In [31]:
for epoch in range(num_epochs):
    for X, y in tqdm(data_loader):
        X = X.cuda()
        y = y.cuda()
        outputs = mlstm.forward(X[:,:-1,:]) #forward pass
        optimizer.zero_grad() #calculate the gradient, manually setting to 0
        
        # obtain the loss function
        # print(outputs.shape, y[:,-1,:].shape)
        loss = criterion(outputs, y[:,-1,:])

        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
    if epoch % 1 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

100%|██████████████████████████████████████████████████████████████████████████████| 4688/4688 [00:47<00:00, 97.89it/s]


Epoch: 0, loss: 0.04944


100%|██████████████████████████████████████████████████████████████████████████████| 4688/4688 [00:47<00:00, 98.17it/s]


Epoch: 1, loss: 0.04503


100%|██████████████████████████████████████████████████████████████████████████████| 4688/4688 [00:48<00:00, 97.56it/s]


...

Epoch: 97, loss: 0.00000


100%|██████████████████████████████████████████████████████████████████████████████| 4688/4688 [00:53<00:00, 88.04it/s]


Epoch: 98, loss: 0.00000


100%|██████████████████████████████████████████████████████████████████████████████| 4688/4688 [00:52<00:00, 89.04it/s]


Epoch: 99, loss: 0.00000


In [32]:
torch.save(mlstm.state_dict(), '../models/muse_w10_v1.pth')

In [ ]:
torch.__version__

In [ ]:
seed = [0. for i in range(111)]

In [ ]:
seed[60] = 1.

In [ ]:
seed_t = torch.tensor(seed, requires_grad=True).float()

In [ ]:
X_tensor_final.shape

In [ ]:
seed_t = torch.reshape(seed_t, (1, 1, -1))

In [ ]:
seed_t.shape

In [ ]:
p = mlstm(seed_t)

In [ ]:
p.shape

In [ ]:
np.argmax(torch.nn.functional.softmax(p, dim=1).detach().numpy())

In [ ]:
with open('../data/chords_reduced/CHORD_TO_EMB.pickle', 'rb') as f: 
    CHORD_TO_EMB = pickle.load(f)

In [ ]:
CHORD_TO_EMB

In [ ]:
with open('../data/chords/CHORD_DICT.pickle', 'rb') as f: 
    CHORD_DICT = pickle.load(f)

In [ ]:
CHORD_DICT[2]

## Music generation

In [ ]:
mlstm = MelodyLSTM(input_size, hidden_size, output_size, num_layers)
mlstm.load_state_dict(torch.load('../models/muse_v1_111ft.pth')) 
mlstm.eval()

In [ ]:
with open('../data/chords/CHORD_DICT.pickle', 'rb') as f: 
    CHORD_DICT = pickle.load(f)

In [ ]:
with open('../data/chords_reduced/CHORD_TO_EMB.pickle', 'rb') as f:
    CHORD_TO_EMB = pickle.load(f)

In [ ]:
def get_chord_from_triad(t1, t2, t3):
    return t1 * 10000 + t2 * 100 + t3

In [ ]:
def generate_melody_for_chord(chord, prev=None, ts=8):
    notes = []
    if prev == None:
        prev = torch.zeros(111)
        prev[(ub-lb) + chord] = 1.
        prev = torch.reshape(prev, (1, 1, -1))
    
    for _ in range(ts):
        p = mlstm(prev)
        # print(torch.nn.functional.softmax(p, dim=1).detach().numpy())
        notes.append(np.argmax(torch.nn.functional.softmax(p, dim=1).detach().numpy()))
        
        prev = torch.zeros(111)
        prev[(ub-lb) + chord] = 1.
        prev[notes[-1]] = 1.
        prev = torch.reshape(prev, (1, 1, -1))

    return notes, prev

In [ ]:
def generate_melody_for_chords(chords):
    notes = []
    prev = None
    for chord in chords: 
        notes_i, prev = generate_melody_for_chord(chord, prev)
        print(len(notes_i))
        notes += notes_i
    return notes

In [ ]:
notes = generate_melody_for_chords([0, 1, 2, 3])

In [ ]:
piano_roll = np.zeros((128, 32))

In [ ]:
for i, note in enumerate(notes):
    piano_roll[note + lb, i] = 100

In [ ]:
np.count_nonzero(piano_roll)

In [ ]:
notes

In [ ]:
piano_roll[31+lb:37+lb]

In [ ]:
piano_roll.T.shape

In [ ]:
# create a PrettyMIDI object
midi = pretty_midi.PrettyMIDI()

# create an instrument object
instrument = pretty_midi.Instrument(program=0)

# add notes to the instrument object
for note_idx, time_slice in enumerate(piano_roll.T):
    note_numbers = np.nonzero(time_slice)[0]
    for note_number in note_numbers:
        note_start = note_idx / 4.0
        note_end = (note_idx + 1) / 4.0
        note_velocity = int(time_slice[note_number])
        note = pretty_midi.Note(
            velocity=note_velocity,
            pitch=note_number,
            start=note_start,
            end=note_end
        )
        instrument.notes.append(note)

# add the instrument object to the MIDI object
midi.instruments.append(instrument)

# write the MIDI object to a file
midi.write('output_test.mid')

In [ ]:
generate_melody_for_chord(4)

In [ ]:
CHORD_DICT[58]